# Procesamientos de imagenes y video

La pr&aacute;ctica requiri&oacute; grabar a una persona al volante con el motor apagado, a la altura de los ojos. El conductor debe alternar su mirada hacia el frente, izquierda, derecha, retrovisor y palanca.
Este proyecto tiene la finalidad de evaluar nuevos videos en tiempo real para determinar el desempeno del detector de PAVs.

In [6]:
# Importando librerias
import pandas as pd
import cv2
import os

In [9]:
# Crear un diccionario con las clases y los valores
d = {
    "FRENTE": ["00:00 - 00:09"],
    "IZQUIERDA": ["00:10- 00:21"],
    "DERECHA": ["00:22-00:29"],
    "RETROVISOR": ["00:30-00:39"],
    "PALANCA": ["00:40-00:49"]
}

# Crear un dataframe con el diccionario
df = pd.DataFrame(d)

# Mostrar el dataframe
print(df)
df.to_csv("C:/Users/ages0/Documents/Ciencia de Datos Para Sensado Inteligente/Practica2_CDSI_ArianaEspinoza/ARIANA.csv")

          FRENTE     IZQUIERDA      DERECHA   RETROVISOR      PALANCA
0  00:00 - 00:09  00:10- 00:21  00:22-00:30  00:31-00:39  00:40-00:49


In [14]:
import cv2
import os

def asegurar_directorio(path):
    if not os.path.exists(path):
        os.makedirs(path)

def segmentar_video_y_guardar_frames(video_path, duraciones_segmentos):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    inicio_frame_actual = 0
    for i, duracion in enumerate(duraciones_segmentos):
        nombre_carpeta_segmento = f'segmento_{i+1}'
        nombre_carpeta_frames = f'{nombre_carpeta_segmento}/frames'
        asegurar_directorio(nombre_carpeta_segmento)
        asegurar_directorio(nombre_carpeta_frames)
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(f'{nombre_carpeta_segmento}/segmento_{i+1}.mp4', fourcc, fps, (int(cap.get(3)), int(cap.get(4))))
        
        num_frames_segmento = int(duracion * fps)
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, inicio_frame_actual)
        
        for j in range(num_frames_segmento):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
            # Guardar el frame actual como imagen en la carpeta de frames
            cv2.imwrite(f'{nombre_carpeta_frames}/frame_{j+1}.jpg', frame)
        
        inicio_frame_actual += num_frames_segmento
        out.release()
    
    cap.release()

# Ejemplo de cómo llamar a la función con duraciones personalizadas
video_path = 'C:/Users/ages0/Desktop/ARIANA.mp4'
duraciones_segmentos = [9, 12, 9, 9, 10]  # Lista de duraciones de segmentos en segundos
segmentar_video_y_guardar_frames(video_path, duraciones_segmentos)
